In [3]:
from overlaps import (
    read_overlaps_from_file,
)
from misclassifications import get_misclassifications, label_misclassifications, LABELERS, groupby_gene
from main import OVERLAP_FILENAME
from schema import DFSchema

overlaps = read_overlaps_from_file(OVERLAP_FILENAME)
misclass_df = get_misclassifications(overlaps)
label_misclassifications(misclass_df)

In [4]:
for labeler in LABELERS:
    print(labeler.summarize_category_count(misclass_df))
    print()

FalsePos_Category
True    133
Name: count, dtype: int64

FalseNeg_Category
True    112
Name: count, dtype: int64

DistToTSSSize_Category
medium    135
large      79
small      31
Name: count, dtype: int64

Top5Gene_Category
MYC        23
PQBP1      14
PLP2       13
PRDX2      13
WDR83OS    10
Name: count, dtype: int64



In [5]:
# Look at MYC classification
total_myc_eg_pairs = len(overlaps[overlaps[DFSchema.TARGET_GENE + DFSchema.CRISPR_SUFFIX] == 'MYC'])
print(f"Misclassified 24/{total_myc_eg_pairs} eg pairs for myc.\n{int(100 * 24/total_myc_eg_pairs)}% incorrect")

Misclassified 24/282 eg pairs for myc.
8% incorrect


In [6]:
# Example queries
print(misclass_df["FalsePos_Category"].value_counts().filter([True]))
print()
print(len(misclass_df.query("FalseNeg_Category == True")))

FalsePos_Category
True    133
Name: count, dtype: int64

112


In [7]:
misclass_df.iloc[0]

index                                                          2
index_crispr                                                 693
index_crispr.1                                               693
chrom_crispr                                               chr17
start_crispr                                            32909510
end_crispr                                              32913691
name_crispr                     SPACA3|chr17:31236528-31240709:*
EffectSize_crispr                                         -0.687
chrTSS_crispr                                              chr17
TargetGeneTSS_crispr                                    32991613
endTSS_crispr                                         32970377.0
TargetGene_crispr                                         SPACA3
IsSignficant_crispr                                         True
pValueAdjusted_crispr                                        NaN
PowerAtEffectSize25_crispr                              0.359854
ValidConnection_crispr   

In [1]:
!python --version

Python 3.10.11
